# R Script for: Continuous foraging behavior shapes patch-leaving decisions in pigeons: A 3D tracking study

Guillermo Hidalgo Gadea

Optimal foraging behavior is a key component of successful adaptations to natural environments. Understanding how animals decide to stay near food or to leave it for another food patch gives us insights into the underlying cognitive mechanisms that govern adaptive behaviors. 3D pose tracking was used to determine how pigeons exploit a 4 square meter arena with two separate platforms (i.e. food patches) whose absolute and relative elevations were manipulated. Detailed kinematic features of foraging and traveling behaviors were quantified using automated video tracking, without a need for manual coding. Our computational approach captured continuous, high-dimensional movement patterns and enabled precise quantification of travel costs between patches. Combined with mixed-effects survival analysis, our detailed behavioral tracking provided unprecedented insight into the moment-by-moment dynamics of patch-leaving decisions of pigeons. As expected from behavior optimization models, our results showed a preference to visit a ground food platform first, and longer latencies to leave an elevated platform. Foraging activity significantly decreased throughout a session, with shorter visits, less pecks per visit, and a decrease in inter-peck variability. However, a mixed-effects Cox regression modeled pigeons' patch-leaving probability, demonstrating that current and cumulative foraging parameters between patches significantly enhanced the model's predictive power beyond patch accessibility (i.e., beyond travel costs). This suggests that pigeons integrate both current environmental cues and their individual foraging history when making patch-leaving decisions. Our findings are discussed in relation to the marginal value theorem and optimal foraging theory.

`updated 22.08.2025`

# 2) Statistical Analysis

In [ ]:
# for data manipulation
library(dplyr)
library(tidyr)
# libraries for plotting
library(ggplot2)
library(gridExtra)
library(ggpattern)
library(ggfortify)
library(scales)
library(ggsignif)
# for mixed effects regression
library(lme4) # nlme
library(emmeans)
library(lmerTest)
library(car)
# for survival analysis
library(survminer)
library(survival) 
library(coxme) #coxph

## Load Dataset

In [ ]:
# load data
data_path <- "K:/ForagingPlatformsArena_local/Analysis/AggregateVisits.csv"
data <- read.csv(data_path, header = TRUE)
data$PID = as.factor(data$PID)
cat("Number of visits recorded by Pigeon and condition:")
table(data$PID, data$condition)

## Refactor Variables

In [ ]:
# reset session_number to start at 1 for each individual PID (exclude habituation count)
min_session_number = aggregate(session ~ PID, data = data, FUN = min)
data = merge(data, min_session_number, by = "PID", suffixes = c("", "_min_task"))
data$session_number_ontask = data$session + 1 - data$session_min_task
data$session_number_ontask = as.factor(data$session_number_ontask)

# reset the session number for each condition
data$session_number_oncond = as.integer(data$session_number_ontask)
data$session_number_oncond[data$session_number_oncond == 5] = 1
data$session_number_oncond[data$session_number_oncond == 6] = 2
data$session_number_oncond[data$session_number_oncond == 7] = 3
data$session_number_oncond[data$session_number_oncond == 8] = 4
data$session_number_oncond[data$session_number_oncond == 9] = 1
data$session_number_oncond[data$session_number_oncond == 10] = 2
data$session_number_oncond[data$session_number_oncond == 11] = 3
data$session_number_oncond[data$session_number_oncond == 12] = 4

# make condition a factor and set '0-0' as reference
data$condition = gsub("75-0", "0-75", data$condition) # TODO check if this is needed
data$condition = as.factor(data$condition)
data$condition = relevel(data$condition, ref = "0-0")

# make location a factor and set 'platform A' as reference
data$location = as.factor(data$location)
data$location = relevel(data$location, ref = "platform A")

# make elevation a factor and set '0' as reference
data$elevation = as.factor(data$elevation)
data$elevation = relevel(data$elevation, ref = "0")

# create interaction term between condition and elevation
data$cond = interaction(data$condition, data$elevation)
data$cond = gsub("0-0.0", "0-0 low", data$cond)
data$cond = gsub("0-75.0", "0-75 low", data$cond)
data$cond = gsub("0-75.750", "0-75 high", data$cond)
data$cond = gsub("75-75.750", "75-75 high", data$cond)
data$cond = as.factor(data$cond)
data$cond = factor(data$cond, levels = c("0-0 low", "0-75 low", "0-75 high", "75-75 high"))
data$cond = relevel(data$cond, ref = "0-0 low")

# make sex a factor and set female as reference
data$sex = as.factor(data$sex)
data$sex = relevel(data$sex, ref = "female")

# reformat covariates
data$age = as.numeric(data$age)
data$normal_weight = as.numeric(data$normal_weight)
data$weight = as.numeric(data$weight)
data$deprivation = 1-data$weight/data$normal_weight
data$performance = as.numeric(data$food_consumed)/60

# check that pigeons forage from both platforms
data$visited =  ifelse(data$depleted_A > 0, 1, 0) + 
                ifelse(data$depleted_B > 0, 1, 0)


In [ ]:
# Reset coding for depleted platforms high and low
# Note that depleted_A + depleted_B != food consumed, when food items were found on the ground

data <- data %>%
  mutate(
    # Create interaction variable
    int = gsub(" ", ".", interaction(elevation, location, condition)),
    
    # Initialize depleted_low and depleted_high
    depleted_low = ifelse(condition == "0-0", depleted_A + depleted_B, 0),
    depleted_high = ifelse(condition == "75-75", depleted_A + depleted_B, 0),
    
    # Handle mixed condition "0-75"
    depleted_low = ifelse(
      condition == "0-75" & location == "platform A" & elevation == "0", depleted_A,
      ifelse(condition == "0-75" & location == "platform A" & elevation == "750", depleted_B,
      ifelse(condition == "0-75" & location == "platform B" & elevation == "0", depleted_B,
      ifelse(condition == "0-75" & location == "platform B" & elevation == "750", depleted_A,
      depleted_low)))),
    
    depleted_high = ifelse(
      condition == "0-75" & location == "platform A" & elevation == "0", depleted_B,
      ifelse(condition == "0-75" & location == "platform A" & elevation == "750", depleted_A,
      ifelse(condition == "0-75" & location == "platform B" & elevation == "0", depleted_A,
      ifelse(condition == "0-75" & location == "platform B" & elevation == "750", depleted_B,
      depleted_high))))
  )

## Correct dataset agains self-transitions

Pigeons often visit a platform, leave it for a short period of time, and return to the same platform to continue foraging on the same platform. We handle these as self-transitions, opposed to transitions between alternative platforms. Importantly, once a pigeon leaves a platform towards the alternative, we consider the aggregate activity on that platform throughout all smaller self-transitions.

Below, we look at "real" visits/transitions as the `cumsum(transition_self == 0)`, group all df rows by the "real" visit and aggregate foraging parameters either as a sum throughout self-transitions, or as the weighted average of e.g. peckrate and variability. Note that these parameters refer to the entire "corrected visit" by the end, when leaving the platform. While other parameters such as travel distance or visit latency refer to the time and distance required to reach the visit prior to the actual visit start.

Censored data of `status == 0` are synthetic, shortened duplicates of real visits, representing cases of active foraging without leaving the platform, e.g. unfinished visits. These help balance the dataset after biased sampling towards finished visits. Censored visits need to be corrected, too. 

`updated 02.04.2025`

In [ ]:
# find matching real-synthetic visit pairs by total_latency
data <- data %>%
  group_by(PID, session_number_oncond, condition) %>%
  # flag synthetic visits matching real visits
  mutate(uncensored_flag = as.integer(status == 1 & (duplicated(total_latency) | duplicated(total_latency, fromLast = TRUE)))) %>%
  # flag all self-transitions corresponging to synthetic visits
  ungroup()

In [ ]:
# first pass with real visits
real_visits <- data %>%
  # remove synthetic visits in conflicting duplicates
  filter(status == 1) %>%
  arrange(PID, session_number_oncond, condition, visit_order) %>%
  mutate(full_visits = cumsum(transition_self == 0)) %>%
  group_by(PID, session_number_oncond, condition, full_visits) %>%
  mutate(
    # aggregate parameters over "real visits"
    total_visit_length = round(sum(visit_length, na.rm = TRUE), 2),
    total_head_disp = round(sum(head_disp, na.rm = TRUE), 2),
    total_peck_count = round(sum(peck_count, na.rm = TRUE), 2),
    num_self_transitions = round(sum(transition_self, na.rm = TRUE), 2),
    self_travel_dist = round(sum(travel_dist[transition_self != 0], na.rm = TRUE), 2),
    # calculate weighted averages for all (self-)visits
    total_peck_rate = round(sum(peck_rate * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPI_rmssd = round(sum(IPI_rmssd * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPD_rmssd = round(sum(IPD_rmssd * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPI_entropy = round(sum(IPI_entropy * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPD_entropy = round(sum(IPD_entropy * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    # calculate relative cumulative parameter as highest cum value
    overpeck_ratio = (max(cum_peck_count, na.rm = TRUE) / (max(cum_peck_count, na.rm = TRUE) + max(alt_cum_peck_count, na.rm = TRUE) + 0.1)) - 0.5,
    ) %>%
  ungroup()

# second pass with synthetic visits
synthetic_visits <- data %>%
  # remove real visits in conflicting duplicates 
  filter(uncensored_flag != 1) %>%
  arrange(PID, session_number_oncond, condition, visit_order) %>%
  mutate(full_visits = cumsum(transition_self == 0)) %>%
  group_by(PID, session_number_oncond, condition, full_visits) %>%
  mutate(
    # aggregate parameters over "real visits"
    total_visit_length = round(sum(visit_length, na.rm = TRUE), 2),
    total_head_disp = round(sum(head_disp, na.rm = TRUE), 2),
    total_peck_count = round(sum(peck_count, na.rm = TRUE), 2),
    num_self_transitions = round(sum(transition_self, na.rm = TRUE), 2),
    self_travel_dist = round(sum(travel_dist[transition_self != 0], na.rm = TRUE), 2),
    # calculate weighted averages for all (self-)visits
    total_peck_rate = round(sum(peck_rate * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPI_rmssd = round(sum(IPI_rmssd * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPD_rmssd = round(sum(IPD_rmssd * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPI_entropy = round(sum(IPI_entropy * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    total_IPD_entropy = round(sum(IPD_entropy * visit_length, na.rm = TRUE) / sum(visit_length, na.rm = TRUE), 2),
    # calculate relative cumulative parameter as highest cum value
    overpeck_ratio = (max(cum_peck_count, na.rm = TRUE) / (max(cum_peck_count, na.rm = TRUE) + max(alt_cum_peck_count, na.rm = TRUE) + 0.1)) - 0.5,
    ) %>%
  ungroup()
# remove real visits already corrected above
synthetic_visits <- subset(synthetic_visits, status == 0)

# Combine the results
corrected_data <- bind_rows(real_visits, synthetic_visits) %>%
arrange(PID, session_number_oncond, condition, visit_order)

In [ ]:
# total number of recorded visits, including censored data
cat("Total number of visits by Pigeon and condition, including", 
    round(prop.table(table(subset(corrected_data, (transition_self == 0))$status))["0"]*100,2), 
    "% censored visits:\n")
print(table(subset(corrected_data, transition_self == 0)$PID, subset(corrected_data, transition_self == 0)$session_number_ontask))

## Filter Dataset

The dataset is first filtered to exclude pigeons that did not consume the food items used during the experiment. 

Second, we filter sessions in which any platform is not visited. Note that given the task design, failure to visit the elevated platform implied a drop in foraging performance to 50% in the 0-75 condition and to 0% in the 75-75 condition, and platform transitions could not be calculated. These individual sessions were removed from further analysis, excluding a total of 18 out of 100 test sessions due to non-visited platforms.

Third, we filter visits without foraging activity to exclude non-foraging use of the platforms, as well as visits shorter than 2 seconds to exclude passing-throughs.

Lastly, we trimm the highest 1% of visit lengths, latencies and peck counts to exclude outliers in heavy tailed distributions.


`updated 02.04.2025`

### Exclude by Task Engagement

Pigeons that do not interact with the foraging setup above a performance criterium are excluded from the sample.

In [ ]:
# subset data per session
data_unique = data[!duplicated(interaction(data$session_number_ontask, data$PID)), ]
color_mapping <- setNames(rainbow(length(unique(data_unique$PID))), unique(data_unique$PID))
shape_mapping <- setNames(rep(1:6, length.out = length(unique(data_unique$PID))), unique(data_unique$PID))

# Plot performance data for individual pigeons over session aggregates
ggplot(data_unique, aes(x = session_number_ontask, y = food_consumed)) +
  geom_boxplot(aes(fill = condition), alpha = .2, show.legend=FALSE) + 
  geom_point(aes(color = PID, shape = PID), size = 5) + 
  geom_line(aes(group = PID, color = PID), linewidth = 1.5) + 
  geom_hline(yintercept = 48, color = 'red')+ geom_hline(yintercept = 30, color = 'red')+
  coord_cartesian(ylim = c(0, 60)) + 
  scale_color_manual(values = color_mapping) +
  scale_shape_manual(values = shape_mapping) +
  labs(
    x = "Session Number",
    y = "Food items consumed",
    color = "Pigeon",
  ) +
  theme_classic() + theme(text = element_text(size = 20))
#ggsave("K:/ForagingPlatformsArena_local/food_consumed_raw.png", width = 21, height = 10, dpi = 600)

In [ ]:
# supress ggplot warnings and messages
suppressMessages(suppressWarnings(
# Plot individual performance data and food deprivation levels over time
for (id in unique(data_unique$PID)) {
  data_subset <- data_unique[data_unique$PID == id, ]
  p <- ggplot(data_subset, aes(x = as.integer(session_number_ontask))) +
    geom_rect(aes(xmin = 4.5, xmax = 8.5, ymin = -Inf, ymax = Inf), fill = "grey", alpha = 0.1) +
    geom_point(aes(y = performance, color = "Performance"), size = 5) + 
    geom_line(aes(y = performance, color = "Performance", group = 1), linewidth = 1.5) +  # Explicit group
    geom_hline(aes(yintercept = 0.25), color = 'red', linetype = "dashed", show.legend = FALSE) +
    geom_point(aes(y = deprivation, color = "Deprivation"), size = 5) + 
    geom_line(aes(y = deprivation, color = "Deprivation", group = 1), linewidth = 1.5) +  # Explicit group
    geom_hline(aes(yintercept = 0.50), color = 'black', linetype = "dashed", show.legend = FALSE) +
    labs(
      x = "Session Number",
      y = "Percent",
      color = "Variable",
      title = paste("Pigeon ID:", id)
    ) + 
    scale_x_continuous(limits = c(1, 12), breaks = 1:12) +
    scale_y_continuous(limits = c(0, 1), breaks = seq(0, 1, 0.1)) +
    scale_color_manual(values = c("Performance" = "black", "Deprivation" = "red")) +
    theme_classic() +
    theme(
      text = element_text(size = 20),
      plot.title = element_text(hjust = 0.5)  # Center the title
    )
  print(p)
  # ggsave(paste0("K:/ForagingPlatformsArena_local/", id, "_performance_deprivation.png"), plot = p, width = 10, height = 3, dpi = 600)

}))



In [ ]:
# flag Pigeons that do not engage with the task, see Figures above
corrected_data$excluded1 = ifelse(corrected_data$PID %in% c("P589", "P791", "P122"), 1, 0)
cat("Flagged a total of", table(corrected_data$excluded1)["1"], "visits from the sample.\n")
cat("Out of N = 12 pigeons, 3 did not show enough interest in the foraging platforms.")
print(table(corrected_data$PID, corrected_data$excluded1))

### Exclude by Platform Visits

In [ ]:
# mark sessions inwhich pigeons do not visit both platforms
corrected_data$excluded2 = ifelse(corrected_data$visited < 2, 1, 0) 

# from the remaining 9 pigeons, 3 never visited the elevated platform in 0-75 and 2 never visited the elevated platform in 75-75
cat("Flagged a total of", table(corrected_data$excluded2)["1"], "visits from the sample, distrubuted as follows:\n")
print(table(corrected_data$PID, corrected_data$excluded2))

### Exclude by Visit Requirements

Visits with `peck_count < 1` are considered non-foraging visits, and visit lengths of `visit_length < 2sec` are considered passing-through and excluded from the dataset.

In [ ]:
# subset data by visit definition of >2sec and >1 pecks
corrected_data$excluded3 = ifelse((corrected_data$total_visit_length < 2 | corrected_data$total_peck_count < 1), 1, 0)
cat("Flagged a total of", table(corrected_data$excluded3)["1"], "visits from the sample.")
print(table(corrected_data$PID, corrected_data$excluded3))


### Excluded by Trimmed Outliers

Outlier correction by trimming, note that outlier correction is applied to the already filtered dataset.

In [ ]:
# apply preious exclusion criteria to calculate outliers
data_trimming <- subset(corrected_data, transition_self == 0 & status == 1 & excluded1 == 0 & excluded2 == 0 & excluded3 == 0)

# calculate 1% trimm of the data for robust estimation
higher_peckcount = quantile(data_trimming$total_peck_count, probs = 0.99)
higher_visit_length = quantile(data_trimming$total_visit_length, probs = 0.99)
higher_latency = quantile(data_trimming$visit_latency, probs = 0.99)

# create mask for trimmed outliers
corrected_data$excluded4 = ifelse(corrected_data$total_peck_count >= higher_peckcount, 1, 0)
corrected_data$excluded4[corrected_data$total_visit_length >= higher_visit_length] = 1
corrected_data$excluded4[corrected_data$visit_latency >= higher_latency] = 1
cat("Flagged a total of", table(corrected_data$excluded4)["1"], "visits from the sample, or",round(prop.table(table(corrected_data$excluded4))["1"]*100,2),"%.\n")

In [ ]:
# plot heavy tailed distributions
p1 <- ggplot(data_trimming, aes(x = visit_latency)) +
  geom_histogram(bins = 100) +
  geom_vline(xintercept = higher_latency, color = "red", linetype = 'dashed') +
  labs(x = "Time Latency (sec)", y = "Raw Frequency") +
  theme_classic() + theme(text = element_text(size = 20),
  plot.title = element_text(size = 20),
  axis.title = element_text(size = 20),
  axis.text = element_text(size = 20),
  legend.title = element_text(size = 20),
  legend.text = element_text(size = 20))
p2 <- ggplot(data_trimming, aes(x = total_visit_length)) +
  geom_histogram(bins = 100) +
  geom_vline(xintercept = higher_visit_length, color = "red", linetype = 'dashed') +
  labs(x = "Visit Length (sec)", y = "Raw Frequency") +
  theme_classic() + theme(text = element_text(size = 20),
  plot.title = element_text(size = 20),
  axis.title = element_text(size = 20),
  axis.text = element_text(size = 20),
  legend.title = element_text(size = 20),
  legend.text = element_text(size = 20))
p3 <- ggplot(data_trimming, aes(x = total_peck_count)) +
  geom_histogram(bins = 100) +
  geom_vline(xintercept = higher_peckcount, color = "red", linetype = 'dashed') +
  labs(x = "Peck Count", y = "Raw Frequency") +
  theme_classic() + theme(text = element_text(size = 20),
  plot.title = element_text(size = 20),
  axis.title = element_text(size = 20),
  axis.text = element_text(size = 20),
  legend.title = element_text(size = 20),
  legend.text = element_text(size = 20))
  
p4 <- ggplot(data_trimming[data_trimming$visit_latency<higher_latency,], aes(x = visit_latency)) +
  geom_histogram(bins = 100) +
  geom_vline(xintercept = median(data_trimming[data_trimming$visit_latency<higher_latency,]$visit_latency), color = "green") +
  labs(x = "Time Latency (sec)", y = "Filtered Frequency") +
  theme_classic() + theme(
  plot.title = element_text(size = 20),
  axis.title = element_text(size = 20),
  axis.text = element_text(size = 20),
  legend.title = element_text(size = 20),
  legend.text = element_text(size = 20))
p5 <- ggplot(data_trimming[data_trimming$total_visit_length<higher_visit_length,], aes(x = total_visit_length)) +
  geom_histogram(bins = 100) +
  geom_vline(xintercept = median(data_trimming[data_trimming$total_visit_length<higher_visit_length,]$total_visit_length), color = "green") +
  labs(x = "Visit Length (sec)", y = "Filtered Frequency") +
  theme_classic() + theme(
  plot.title = element_text(size = 20),
  axis.title = element_text(size = 20),
  axis.text = element_text(size = 20),
  legend.title = element_text(size = 20),
  legend.text = element_text(size = 20))
p6 <- ggplot(data_trimming[data_trimming$total_peck_count<higher_peckcount,], aes(x = total_peck_count)) +
  geom_histogram(bins = 100) +
  geom_vline(xintercept = median(data_trimming[data_trimming$total_peck_count<higher_peckcount,]$total_peck_count), color = "green") +
  labs(x = "Peck Count", y = "Filtered Frequency") +
  theme_classic() + theme(
  plot.title = element_text(size = 20),
  axis.title = element_text(size = 20),
  axis.text = element_text(size = 20),
  legend.title = element_text(size = 20),
  legend.text = element_text(size = 20))

# create data filtering grid
grid = grid.arrange(p1, p2, p3, p4, p5, p6, ncol = 3)
#ggsave("K:/ForagingPlatformsArena_local/filtered_distributions.png", plot = grid, width = 16, height = 10, dpi = 600)


## Create Datasets

In [ ]:
# Calculate number of visits excluded by all criteria and convert to 0/1
corrected_data$excluded_flag <- as.integer(
  corrected_data$excluded1 == 1 | corrected_data$excluded2 == 1 | 
  corrected_data$excluded3 == 1 | corrected_data$excluded4 == 1 )

# exclude non-engaging pigeons
data_filtered = subset(corrected_data, excluded1 == 0)

# exclude pigeons that never visited both platforms
data_filtered = subset(data_filtered, excluded2 == 0)

# exclude visits that are too short or have too few pecks
data_filtered = subset(data_filtered, excluded3 == 0)

# exclude outliers by 1% trimming
data_filtered = subset(data_filtered, excluded4 == 0)
data_filtered$PID = droplevels(data_filtered$PID)

cat("Flagged a combined total of", table(corrected_data[(corrected_data$status == 1 & corrected_data$transition_self == 0),]$excluded_flag)["1"], "visits from the sample, or",round(prop.table(table(corrected_data[(corrected_data$status == 1 & corrected_data$transition_self == 0),]$excluded_flag))["1"]*100,2),"%.\n")

In [ ]:
data_filtered$num_transitions = data_filtered$visit_order_noself - 1

# Normalize covariates
data_filtered$z_num_transitions <- scale(data_filtered$num_transitions, center = TRUE, scale = TRUE)
data_filtered$z_num_self_transitions = scale(data_filtered$num_self_transitions, center = TRUE, scale = TRUE)

# calculate group mean centered num_self_transitions
data_filtered <- data_filtered %>%
    group_by(cond) %>%
    mutate(
        gmz_num_self_transitions = scale(num_self_transitions, center = TRUE, scale = TRUE),
        gmz_num_transitions = scale(num_transitions, center = TRUE, scale = TRUE),
        gmz_travel_dist = scale(travel_dist, center = TRUE, scale = TRUE)
    ) %>%
    ungroup()

data_filtered$z_session_number_oncond = scale(data_filtered$session_number_oncond, center = TRUE, scale = TRUE)
data_filtered$z_age = scale(data_filtered$age, center = TRUE, scale = TRUE)
data_filtered$z_normal_weight = scale(data_filtered$normal_weight, center = TRUE, scale = TRUE)
data_filtered$z_deprivation = scale(data_filtered$deprivation, center = TRUE, scale = TRUE)
data_filtered$z_performance = scale(data_filtered$performance, center = TRUE, scale = TRUE)

data_filtered$z_visit_latency = scale(data_filtered$visit_latency, center = TRUE, scale = TRUE)
data_filtered$z_travel_time = scale(data_filtered$travel_time, center = TRUE, scale = TRUE)
data_filtered$z_travel_dist = scale(data_filtered$travel_dist, center = TRUE, scale = TRUE)
data_filtered$z_self_travel_dist = scale(data_filtered$self_travel_dist, center = TRUE, scale = TRUE)
data_filtered$z_total_visit_length = scale(data_filtered$total_visit_length, center = TRUE, scale = TRUE)

data_filtered$z_total_peck_count = scale(data_filtered$total_peck_count, center = TRUE, scale = TRUE)
data_filtered$z_total_peck_rate = scale(data_filtered$total_peck_rate, center = TRUE, scale = TRUE)
data_filtered$z_total_head_disp = scale(data_filtered$total_head_disp, center = TRUE, scale = TRUE)
data_filtered$z_total_IPI_entropy = scale(data_filtered$total_IPI_entropy, center = TRUE, scale = TRUE)
data_filtered$z_total_IPI_rmssd = scale(data_filtered$total_IPI_rmssd, center = TRUE, scale = TRUE)
data_filtered$z_total_IPD_entropy = scale(data_filtered$total_IPD_entropy, center = TRUE, scale = TRUE)
data_filtered$z_total_IPD_rmssd = scale(data_filtered$total_IPD_rmssd, center = TRUE, scale = TRUE)
data_filtered$z_overpeck_ratio = scale(data_filtered$overpeck_ratio, center = TRUE, scale = TRUE)


In [ ]:
# firts visit, n = 75
data_first_visit <- subset(data_filtered, (status ==1 &  transition_self == 0 & visit_order == 1))

# second visit, n = 75
data_second_visit <- subset(data_filtered, (status == 1 & transition_self == 0 & visit_order_noself == 2 & current_order == 1))

# first elevated visit, n = 42
data_first_visit_elevated <- bind_rows(subset(data_second_visit, condition == "0-75"), subset(data_first_visit, condition == "75-75"))
data_first_visit_elevated$PID = droplevels(data_first_visit_elevated$PID)

# depletion data, n = 87
data_depletion <- corrected_data %>%
    filter(status == 1 & transition_self == 0 & excluded1 == 0 & excluded3 == 0 & excluded4 == 0) %>%
    group_by(PID, session_number_oncond, condition) %>%
    mutate(
        LH_diff = ifelse(condition == "0-75", 
                        depleted_low - depleted_high, 
                        depleted_A - depleted_B)
    )%>%
    # keep only single row per session
    slice(which.max(visit_order_noself)) %>%
    ungroup()

# performance data, n = 81
# TODO what about performance < 48?
data_performance <- data_filtered %>%
    filter(status == 1 & transition_self == 0) %>%
    group_by(PID, session_number_oncond, condition) %>%
    mutate(
        LH_diff = ifelse(condition == "0-75", 
                        depleted_low - depleted_high, 
                        depleted_A - depleted_B)
    )%>%
    # keep only single row per session
    slice(which.max(visit_order_noself)) %>%
    ungroup()


# total visit count per platform per session, n = 162
data_visit_count_platform <- data_filtered %>%
    filter(status == 1 & transition_self == 0) %>%
    group_by(PID, session_number_oncond, condition, location) %>%
    mutate(
        self_transitions_count = sum(num_self_transitions)
    )%>%
    # keep only single row per platform per session
    slice(which.max(visit_order_noself)) %>%
    ungroup()

# total visit count per session, n = 81
data_visit_count <- data_visit_count_platform %>%
    group_by(PID, session_number_oncond, condition) %>%
    # keep only single row per session
    slice(which.max(visit_order_noself)) %>%
    ungroup()

# real visits data, n = 1744, 37% censored
data_visits <- subset(data_filtered, transition_self == 0)

# real visits without censoring, n = 1101
data_visits_finished <- subset(data_visits, status == 1)

# prepare copy for residual correction
data_visits_res <- data_visits

#see section 3.3) Split dataset
k = 8 
# first visits
data_visits_first = subset(data_visits, num_transitions <= k)

# last visits
data_visits_last <- data_visits %>%
    group_by(PID, session_number_oncond, condition) %>%
    mutate(
        last_k_visit = ifelse(visit_order_noself > (max(visit_order_noself) - k), 1, 0)
    ) %>%
    ungroup() %>%
    filter(last_k_visit == 1) %>%
    select(-last_k_visit) %>%
    group_by(PID, session_number_oncond, condition) %>%
    mutate(
        last_visit_order = (max(visit_order_noself) - k + 1) - visit_order_noself,
        num_transitions_corr = visit_order_noself - (max(visit_order_noself) - k - 1),
    ) %>%
    ungroup()


# 3) Results

# 3.1 Engagement with Setup

## First visit Latencies

In [ ]:
boxplot(data_first_visit$total_latency ~ data_first_visit$condition, main = "First Visit Latency", xlab = "Condition", ylab = "Latency (sec)")

In [ ]:
# median time latency first visit
data = data_first_visit
med = quantile(data$visit_latency, probs = 0.5)
q1 = quantile(data$visit_latency, probs = 0.25)
q3 = quantile(data$visit_latency, probs = 0.75)
iqr = q3 - q1
paste("Average First Visit latency Mdn = ", med, "sec, IQR = ", iqr, "sec")

# difference between conditions
kruskal.test(visit_latency ~ condition, data)

In [ ]:
# median time latency first visit elevated platform
data = data_first_visit_elevated
med = quantile(data$total_latency[data$condition == "75-75"], probs = 0.5)
q1 = quantile(data$total_latency[data$condition == "75-75"], probs = 0.25)
q3 = quantile(data$total_latency[data$condition == "75-75"], probs = 0.75)
iqr = q3 - q1
cat("Elevated first Visit latency for 75-75 Mdn = ", med, "sec, IQR = ", iqr, "sec \n")

med = quantile(data$total_latency[data$condition == "0-75"], probs = 0.5)
q1 = quantile(data$total_latency[data$condition == "0-75"], probs = 0.25)
q3 = quantile(data$total_latency[data$condition == "0-75"], probs = 0.75)
iqr = q3 - q1
cat("Elevated first Visit latency for 0-75 Mdn = ", med, "sec, IQR = ", iqr, "sec \n")

# difference between conditions
kruskal.test(total_latency ~ condition, data)

# median time spent on previous platform
med = quantile(data$alt_cum_visit_length[data$condition == "0-75"], probs = 0.5)
q1 = quantile(data$alt_cum_visit_length[data$condition == "0-75"], probs = 0.25)
q3 = quantile(data$alt_cum_visit_length[data$condition == "0-75"], probs = 0.75)
iqr = q3 - q1
cat("Previous cumulative visit length for 0-75 Mdn = ", med, "sec, IQR = ", iqr, "sec \n")


## Platform preference

In [ ]:
boxplot(data_first_visit$total_latency ~ data_first_visit$condition, main = "First Visit Latency", xlab = "Pigeon ID", ylab = "Latency (sec)")

In [ ]:
# on the ground
data = subset(data_first_visit, condition == "0-0")
binom.test(table(data$location), alternative = "two.sided")

# latency to first platform location 
#wilcox.test(data$visit_latency ~ data$location, alternative = "two.sided")

In [ ]:
# elevated
data = subset(data_first_visit, condition == "75-75")
binom.test(table(data$location), alternative = "two.sided")

# latency to first platform location 
#wilcox.test(data$visit_latency ~ data$location, alternative = "two.sided")


## Latency second visit

In [ ]:
boxplot(data_second_visit$total_latency ~ data_second_visit$condition, main = "Second Visit Latency", xlab = "Condition", ylab = "Latency (sec)")

In [ ]:
# median time latency second visit
data = data_second_visit
med = quantile(data$total_latency, probs = 0.5)
q1 = quantile(data$total_latency, probs = 0.25)
q3 = quantile(data$total_latency, probs = 0.75)
iqr = q3 - q1
cat("Average Second Visit latency Mdn = ", med, "sec, IQR = ", iqr, "sec \n")
# Mdn = 10.18 sec, IQR = 3.14 sec

data_second_visit %>%
  group_by(condition) %>%
  summarise(median = quantile(total_latency, probs = 0.5), IQR = quantile(total_latency, probs = 0.75)-quantile(total_latency, probs = 0.25))


In [ ]:
# Perform a Wilcoxon rank-sum test
wilcox.test(total_latency ~ condition, data = subset(data, condition != "75-75"))
wilcox.test(total_latency ~ condition, data = subset(data, condition != "0-0"))
wilcox.test(total_latency ~ condition, data = subset(data, condition != "0-75"))


# 3.2 Foraging Differences

Compare per-session foraging parameters such as `performance` and `number of transitions` between conditions, i.e. travel cost.

In [ ]:
table(data_performance$depleted_A, data_performance$depleted_B)

## Model 1: Performance

In [ ]:
# food consumed
data = data_performance
model1 <- lmer(food_consumed ~ condition + z_session_number_oncond + z_age + z_deprivation + z_normal_weight + (1 | PID), data)
summary(model1)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(model1))
hist(residuals(model1), breaks = 50)
plot(fitted(model1), residuals(model1))

# posthoc pairwise
emm_condition <- emmeans(model1, ~ condition)
pairwise_condition <- contrast(emm_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)



In [ ]:
emm_df <- as.data.frame(emm_condition)
scale = 2.5*max(emm_df$upper.CL - emm_df$lower.CL)
low = mean(emm_df$emmean) - scale/2
high = mean(emm_df$emmean) + scale/2

In [ ]:
# Plot the EMMs
ggplot(emm_df, aes(x = condition, y = emmean, fill = condition, color = condition)) +
  geom_errorbar(aes(ymin = lower.CL, ymax = upper.CL), width = 0.5, color = "black") +
  geom_point(shape = 18, size = 10, show.legend = FALSE) +
  scale_color_brewer(palette = "Set1") +
  coord_cartesian(ylim = c(low, high)) +
  
  scale_y_continuous(
    breaks = seq(low, 60, length.out = 5),  # Adjust the number of breaks as needed
    labels = scales::number_format(accuracy = 5)  # Format the labels
  ) +
  labs(
    x = "Platform condition",
    y = "Food items consumed",
  ) +
  theme_classic() + theme(text = element_text(size = 20)) +
  theme(aspect.ratio = 1)

ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure3A.png", height = 100, units = "mm", dpi = 600)

In [ ]:
options(repr.plot.height=10)
ggplot(data, aes(x = session_number_ontask, y = food_consumed, fill = condition)) +
  geom_boxplot() +
  scale_fill_brewer(palette = "Set1") +
  labs(
    x = "Session Number",
    y = "Food items consumed"
  ) +
  theme_classic() + theme(text = element_text(size = 20)) + theme(aspect.ratio = 1) +
  facet_wrap(~condition, ncol = 1, scales = "free_x", labeller = label_both) +
  theme(strip.background = element_blank(), strip.text = element_text(size = 20), legend.position = "none")
#ggsave("K:/ForagingPlatformsArena_local/food_consumed_filt.png", width = 8, height = 10, dpi = 600)

In [ ]:
options(repr.plot.height=5)

ggplot(data, aes(x = condition, y = food_consumed, fill = condition)) +
  geom_boxplot(notch = TRUE, show.legend = FALSE) +
  scale_fill_brewer(palette = "Set1") +
  labs(
    x = "Condition",
    y = "Food items consumed",
    fill = "Condition"
  ) +
  theme_classic() + theme(text = element_text(size = 20))+
  theme(aspect.ratio = 1)
#ggsave("K:/ForagingPlatformsArena_local/food_consumed_filt.png", width = 8, height = 10, dpi = 600)

## Model 2: Depletion difference

In [ ]:
# median and IQR of depletion difference
paste("Median platform difference in consumed food items Mdn = ", quantile(abs(data_performance$LH_diff), probs = 0.5), 
", IQR = ", quantile(abs(data_performance$LH_diff), probs = 0.75) - quantile(abs(data_performance$LH_diff), probs = 0.25), "")

# kruskal wallis
kruskal.test(abs(LH_diff) ~ condition, data_performance)

subset(data_performance, condition == "0-75") %>%
    group_by(LH_diff_sign = case_when(
        LH_diff < 0 ~ "LH_diff < 0",
        LH_diff == 0 ~ "LH_diff == 0",
        LH_diff > 0 ~ "LH_diff > 0"
    )) %>%
    summarise(
        count = n(),
        median = quantile(abs(LH_diff), probs = 0.5),
        IQR = quantile(abs(LH_diff), probs = 0.75) - quantile(abs(LH_diff), probs = 0.25)
    )
table(subset(data_performance, condition == "0-75")$LH_diff)

In [ ]:
options(repr.plot.height=10)

ggplot(data, aes(x = session_number_ontask, y = abs(LH_diff), fill = condition)) +
  geom_boxplot() +
  scale_fill_brewer(palette = "Set1") +
  labs(
    x = "Session Number",
    y = "Patch depletion difference",
    fill = "Condition"
  ) + 
  theme_classic() + theme(text = element_text(size = 20)) + theme(aspect.ratio = 1) +
  facet_wrap(~condition, ncol = 1, scales = "free_x", labeller = label_both) +
  theme(strip.background = element_blank(), strip.text = element_text(size = 20), legend.position = "none")
#ggsave("K:/ForagingPlatformsArena_local/food_consumed_diff_filt.png", width = 8, height = 10, dpi = 600)


## Model 3: Transitions per condition

In [ ]:
# number of transitions per session
data = data_visit_count
med <- quantile(data$num_transitions, probs = 0.5)
q1 <- quantile(data$num_transitions -1, probs = 0.25)
q3 <- quantile(data$num_transitions -1, probs = 0.75)
iqr <- q3 - q1
paste("Transitions per session Mdn = ", med, "transitions, IQR = ", iqr, "transitions")


In [ ]:
# number of transitions by condition
# NOTE previous analysis had sudo-replication considering both platforms
data = data_visit_count
model3 <- lmer(num_transitions ~ condition + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(model3)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(model3))
hist(residuals(model3), breaks = 50)
plot(fitted(model3), residuals(model3))

# posthoc
emm_condition <- emmeans(model3, ~ condition)
pairwise_condition <- contrast(emm_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)



In [ ]:
ggplot(data, aes(x = condition, y = num_transitions, fill = condition)) +
  geom_boxplot(notch = TRUE) +
  scale_fill_brewer(palette = "Set1") +
  labs(
    x = "Condition",
    y = "Number of transitions",
    fill = "Condition"
  ) +
  theme_classic() + theme(text = element_text(size = 20)) +
  theme(aspect.ratio=1)
#ggsave("K:/ForagingPlatformsArena_local/total_transition_count.png", width = 8, height = 5, dpi = 600)


In [ ]:
emm_df <- as.data.frame(emm_condition)
scale = 2.5*max(emm_df$upper.CL - emm_df$lower.CL)
low = mean(emm_df$emmean) - scale/2
high = mean(emm_df$emmean) + scale/2

In [ ]:
# Plot the EMMs
ggplot(emm_df, aes(x = condition, y = emmean, fill = condition, color = condition)) +
  geom_errorbar(aes(ymin = lower.CL, ymax = upper.CL), width = 0.5, color = "black") +
  geom_point(shape = 18, size = 10, show.legend = FALSE) +
  scale_color_brewer(palette = "Set1") +
  coord_cartesian(ylim = c(low, high)) +
  labs(
    x = "Platform condition",
    y = "Number of transitions",
    fill = "Condition"
  ) +
  scale_y_continuous(
    breaks = seq(low, 28, length.out = 5),  # Adjust the number of breaks as needed
    labels = scales::number_format(accuracy = 5)  # Format the labels
  ) +
  theme_classic() + theme(text = element_text(size = 20))+
  theme(aspect.ratio = 1)

  ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure3B.png", height = 100, units = "mm", dpi = 600)

## Model 4: Self-transitions per condition

In [ ]:
# number of self transitions per platform per session
data = data_visit_count_platform
med <- quantile(data$self_transitions_count, probs = 0.5)
q1 <- quantile(data$self_transitions_count, probs = 0.25)
q3 <- quantile(data$self_transitions_count, probs = 0.75)
iqr <- q3 - q1
paste("Self-transitions Mdn = ", med, "self-transitions, IQR = ", iqr, "self-transitions")

In [ ]:
boxplot(data_visit_count_platform$self_transitions_count ~ data_visit_count_platform$cond, main = "Self-Transitions", xlab = "Condition", ylab = "Self-Transitions")

In [ ]:
# number of self transitions per platform by condition
data = data_visit_count_platform
model4 <- lmer(self_transitions_count ~ cond + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(model4)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(model4))
hist(residuals(model4), breaks = 50)
plot(fitted(model4), residuals(model4))

# posthoc
emm_condition <- emmeans(model4, ~ cond)
pairwise_condition <- contrast(emm_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)


In [ ]:
ggplot(data, aes(x = elevation, y = self_transitions_count, fill = condition)) +
  geom_boxplot(notch = TRUE) +
  scale_fill_brewer(palette = "Set1") +
  labs(
    x = "Elevation (mm)",
    y = "Number of self-transition",
    fill = "Condition"
  ) +
  theme_classic() + theme(text = element_text(size = 20)) +
  theme(aspect.ratio = 1)
#ggsave("K:/ForagingPlatformsArena_local/total_self_transition_elevation.png", width = 5, height = 5, dpi = 600)

In [ ]:
emm_df <- as.data.frame(emm_condition)
emm_df$condition <- c("0-0", "0-75", "0-75", "75-75")
emm_df$elevation <- c("0 mm", "0 mm", "750 mm", "750 mm")
scale = 2.5*max(emm_df$upper.CL - emm_df$lower.CL)
low = mean(emm_df$emmean) - scale/2
high = mean(emm_df$emmean) + scale/2

In [ ]:
# Plot the EMMs
ggplot(emm_df, aes(x = factor(elevation), y = emmean, color = condition)) +
  geom_errorbar(aes(ymin = lower.CL, ymax = upper.CL, group = condition), width = 0.5, position = position_dodge(width = 0.9), color = "black") +
  geom_point(shape = 18, size = 10, position = position_dodge(width = 0.9), show.legend = TRUE) +
  labs(
    x = "Platform elevation",
    y = "Number of self-transitions",
    fill = "Condition"
  ) +
  scale_color_brewer(palette = "Set1", guide = guide_legend(override.aes = list(shape= 16, size = 6))) +
  coord_cartesian(ylim = c(low, high)) +
  scale_x_discrete(
    breaks = c("0 mm", "750 mm"),
    labels = c("0 mm", "750 mm"),
    drop = FALSE
  ) +  
  scale_y_continuous(
    breaks = seq(low, 28, length.out = 5),  # Adjust the number of breaks as needed
    labels = scales::number_format(accuracy = 5)  # Format the labels
  ) +
  theme_classic() + theme(text = element_text(size = 20))+
  theme(aspect.ratio = 1)

  ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure3C.png", height = 100, units = "mm", dpi = 600)

## Time away from platform

In [ ]:
# subset data for status
data = subset(data_visits, status == 1)

# total latency marks the absolute beginning of each visit since the start of the session
ggplot(data, aes(x = total_latency + total_visit_length, y = visit_order_noself, color = condition)) +
  geom_point(size = 1) +
  geom_smooth(method = "loess", se = TRUE, size = 2) +
  scale_x_continuous(limits = c(0, 1500)) +
  labs(
    x = "Session time (sec)",
    y = "Number of visits",
    color = "Condition"
  ) +
  theme_classic() + theme(text = element_text(size = 20)) +  
  theme(aspect.ratio = 1/2)+
  scale_color_brewer(palette = "Set1")

In [ ]:
# plot cumulative time on platform
ggplot(data, aes(x = total_latency + total_visit_length, y = cum_visit_length + alt_cum_visit_length, color = condition)) +
  scale_x_continuous(limits = c(0, 1500)) +
  scale_y_continuous(limits = c(0, 1500)) +
  geom_point(size = 1) +
  geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "black") +
  geom_smooth(method = "loess", se = TRUE, size = 2) +
  labs(
    x = "Time (sec)",
    y = "Cum. time on platform (sec)",
    color = "Condition"
  ) +
  theme_classic() + theme(text = element_text(size = 20)) +  
  theme(aspect.ratio = 1/2)+
  scale_color_brewer(palette = "Set1")


In [ ]:
# TODO 
# add bar graph of time away from platform over time

In [ ]:
# overall time on platform
total_time_on_platform = aggregate(total_visit_length ~ PID + session_number_ontask, data, FUN = sum)[, 3]
median_onplatform = quantile(total_time_on_platform, probs = 0.5)
q1 = quantile(total_time_on_platform, probs = 0.25)
q3 = quantile(total_time_on_platform, probs = 0.75)
iqr = q3 - q1
percent_med = median_onplatform/(20*60)*100
paste("Overall time on platform Mdn = ", median_onplatform, "sec, IQR = ", iqr, "sec, ", round(percent_med, 2), "%")

# transition time
travel_time = aggregate(travel_time ~ PID + session_number_ontask, data, FUN = sum)[, 3]
median_travel = quantile(travel_time, probs = 0.5)
q1 = quantile(travel_time, probs = 0.25)
q3 = quantile(travel_time, probs = 0.75)
iqr = q3 - q1
percent_med = median_travel/(20*60)*100
paste("Transition time Mdn = ", median_travel, "sec, IQR = ", iqr, "sec, ", round(percent_med, 2), "%")    


In [ ]:
# first 10 minutes
total_time_on_platform = aggregate(total_visit_length ~ PID + session_number_ontask , data = subset(data, total_latency <= 600), FUN = sum)[, 3]
median = quantile(total_time_on_platform, probs = 0.5)
q1 = quantile(total_time_on_platform, probs = 0.25)
q3 = quantile(total_time_on_platform, probs = 0.75)
iqr = q3 - q1
percent_med = median/(median_onplatform)*100
paste("First 10min time on platform Mdn= ", median, "sec, IQR = ", iqr, "sec, ", round(percent_med, 2), "%")    

travel_time = aggregate(travel_time ~ PID + session_number_ontask, data = subset(data, total_latency <= 600), FUN = sum)[, 3]
median = quantile(travel_time, probs = 0.5)
q1 = quantile(travel_time, probs = 0.25)
q3 = quantile(travel_time, probs = 0.75)
iqr = q3 - q1
percent_med = median/(median_travel)*100
paste("First 10min travel time Mdn= ", median, "sec, IQR = ", iqr, "sec, ", round(percent_med, 2), "%")    


# last 10 minutes
total_time_on_platform = aggregate(total_visit_length ~ PID + session_number_ontask , data = subset(data, total_latency > 600), FUN = sum)[, 3]
median = quantile(total_time_on_platform, probs = 0.5)
q1 = quantile(total_time_on_platform, probs = 0.25)
q3 = quantile(total_time_on_platform, probs = 0.75)
iqr = q3 - q1
percent_med = median/(median_onplatform)*100
paste("Last 10min time on platform Mdn= ", median, "sec, IQR = ", iqr, "sec, ", round(percent_med, 2), "%") 

travel_time = aggregate(travel_time ~ PID + session_number_ontask, data = subset(data, total_latency > 600), FUN = sum)[, 3]
median = quantile(travel_time, probs = 0.5)
q1 = quantile(travel_time, probs = 0.25)
q3 = quantile(travel_time, probs = 0.75)
iqr = q3 - q1
percent_med = median/(median_travel)*100
paste("Last 10min travel time Mdn= ", median, "sec, IQR = ", iqr, "sec, ", round(percent_med, 2), "%")    


# (EXCLUDED) Cost of Travel

In [ ]:
# calculate average travel times per condition
data_visits_finished %>%
  group_by(cond) %>%
  summarise(
    median_travel_time = quantile(travel_time, probs = 0.5),
    IQR_travel_time = quantile(travel_time, probs = 0.75) - quantile(travel_time, probs = 0.25),
    median_travel_dist = quantile(travel_dist, probs = 0.5)/1000,
    IQR_travel_dist = (quantile(travel_dist, probs = 0.75) - quantile(travel_dist, probs = 0.25))/1000
  )


## Model 5: Travel distance

In [ ]:
# travel distance
data = data_visits_finished
model5 <- lmer(log(travel_dist) ~ cond + z_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(model5)

emms_condition <- emmeans(model5, ~ cond, type = "response")
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)
log(summary(pairwise_condition)$ratio)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(model5))
hist(residuals(model5), breaks = 50)
plot(fitted(model5), residuals(model5))

plot(emms_condition) +
    labs(
        y = "Condition",
        x = "Travel Distance EMM [mm]",
    ) +
    #scale_x_continuous(limits = c(-0.7, 0.7)) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )

## Model 6: Travel time

In [ ]:
# better residuals with log
data = data_visits_finished
model6 <- lmer(log(travel_time) ~ cond + z_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data = data_visits)
summary(model6)

emms_condition <- emmeans(model6, ~ cond, type = "response")
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)
log(summary(pairwise_condition)$ratio)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(model6))
hist(residuals(model6), breaks = 50)
plot(fitted(model6), residuals(model6))

plot(emms_condition) +
    labs(
        y = "Condition",
        x = "Travel Time EMM [sec]",
    ) +
    #scale_x_continuous(limits = c(-0.7, 0.7)) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )

# 3.3 Cox Regression

Visit length or time on platform can be re-interpreted as latency to leave a platform, i.e. time-to-event, which can then be operationalized as a survival analysis.

Ideally, the survival analysis controls for individual differences as rondom intercept and all the other session and animal covariates, see `coxme` below. Since `survfit` can not handle mixed effect, we use model residuals from a linear model as the corrected time to event for the categorical kaplan meier curves. 

## Stepwise Model Building

Baseline models include individual animal characteristics such as age, performance, deprivation etc, as well as task design covariates infuelncing visit length such as visit order and session number.

The second model step includes foraging parameters in the current platform, such as travel distance to the current platform, the number of previous self-visits on the platform, the peck rate, and pecking variability. Variability as RMSSD captures the magnitude of the variability in inter-peck-interval and inter-peck-distances. 

The final model step includes the cumulative or historical foraging ratio between platforms, i.e. overpecking

In [ ]:
# feature correlation
data = data_visits
cor(data[c("total_visit_length", "gmz_num_transitions", "gmz_num_self_transitions", "gmz_travel_dist", "z_travel_time", "z_total_peck_rate", "z_total_head_disp", "z_total_IPD_entropy", "z_total_IPI_entropy", "z_total_IPI_rmssd", "z_total_IPD_rmssd", "z_overpeck_ratio")])

# peck_rate and IPI_entropy seem to cancel each other out, on themselves they have no effect, but together they show strong opposite effects


In [ ]:
# full dataset, no split
data = data_visits

# base model
foraging_cox_0 <- coxme(Surv(total_visit_length, status) ~ 1 + (1 | PID), data)

# covariates
foraging_cox_1 <- coxme(Surv(total_visit_length, status) ~ 1 + cond + gmz_travel_dist + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + (1 | PID), data)

# foraging parameters
foraging_cox_2 <- coxme(Surv(total_visit_length, status) ~ 1 + cond + gmz_travel_dist + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + gmz_num_self_transitions + z_total_peck_rate + z_total_IPI_rmssd + z_total_IPD_rmssd + (1 | PID), data)

# cumulative foraging parameters
foraging_cox_3 <- coxme(Surv(total_visit_length, status) ~ 1 + cond + gmz_travel_dist + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + gmz_num_self_transitions + z_total_peck_rate + z_total_IPI_rmssd + z_total_IPD_rmssd + z_overpeck_ratio + (1 | PID), data)


In [ ]:
# stepwise cox regression
anova(foraging_cox_0, foraging_cox_1, foraging_cox_2, foraging_cox_3)
# Lower AIC values indicate a better model. It balances model fit and complexity, penalizing models with more parameters.
aic_values <- AIC(foraging_cox_0, foraging_cox_1, foraging_cox_2, foraging_cox_3)
aic_values$Delta_AIC <- c(0,diff(aic_values$AIC))
print(aic_values)

# Lower BIC values indicate a better model. BIC imposes a stronger penalty for models with more parameters compared to AIC, making it more conservative.
bic_values <- BIC(foraging_cox_0, foraging_cox_1, foraging_cox_2, foraging_cox_3)
bic_values$Delta_BIC <- c(0,diff(bic_values$BIC))
print(bic_values)

# check multicollinearity of models
vif(foraging_cox_3)


## Model 7: Cox Model

In [ ]:
# model results
summary(foraging_cox_3)

# get 95% CI for HR
exp(confint(foraging_cox_3))

# main effects
Anova(foraging_cox_3, type = "III") 

# post hoc tests for condition
emm_condition <- emmeans(foraging_cox_3, ~ cond, type = "link")
pairwise_condition <- contrast(emm_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)

plot(emm_condition) +
    labs(
        y = "Platform",
        x = "Hazard EMM",
        color = "Comparison"
    ) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )

# save plot
ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4C.png", height = 100, units = "mm", dpi = 600)


## Visualization

In [ ]:
# generate model predictions from foraging_cox_3 for
# condition

# travel distance cutoff
# num transitions cutoff
# peck rate cutoff
# overpecking cutoff


In [ ]:
# Create residual-corrected time to event
data = data_visits
model_residuals <- lm(log(total_visit_length) ~ 1 + PID + gmz_travel_dist + gmz_num_transitions + gmz_num_self_transitions + z_session_number_oncond + z_age + z_normal_weight + z_performance + z_deprivation + z_total_peck_rate + z_total_IPI_rmssd + z_total_IPD_rmssd + overpeck_ratio, data)

# Add residuals to predicted values to ensure positive times
data$times_res <- exp(residuals(model_residuals) - min(residuals(model_residuals)))
data$times_pred <- predict(model_residuals)
# Plot hazard curves for each condition
km_fit_adjusted <- survfit(Surv(times_res, status) ~ cond, data = subset(data, times_res < 120))

# Convert survfit object to a dataframe for ggplot, including confidence intervals
km_data <- data.frame(
  time = km_fit_adjusted$time,
  diff = c(NA, diff(km_fit_adjusted$cumhaz) / diff(km_fit_adjusted$time)), # Calculate the hazard rate as the change in cumulative hazard over time
  surv = km_fit_adjusted$surv,
  lower = km_fit_adjusted$lower,
  upper = km_fit_adjusted$upper,
  hazard = km_fit_adjusted$cumhaz,
  hazard_lower = pmax(0, km_fit_adjusted$cumhaz - 1.96 * km_fit_adjusted$std.chaz),
  hazard_upper = km_fit_adjusted$cumhaz + 1.96 * km_fit_adjusted$std.chaz,
  strata = rep(levels(data$cond), km_fit_adjusted$strata)
)

# Normalize and smooth the instantaneous hazard
km_data$diff_normalized <- NA
for (stratum in unique(km_data$strata)) {
  subset_data <- km_data[km_data$strata == stratum, ]
  # Smooth the instantaneous hazard using LOESS
  smoothed_diff <- loess(diff ~ time, data = subset_data, span = 0.2)$fitted
  # Normalize the smoothed instantaneous hazard
  max_smoothed_diff <- max(smoothed_diff, na.rm = TRUE)
  km_data$diff_normalized[km_data$strata == stratum] <- smoothed_diff / max_smoothed_diff
}

p1 <- ggplot(km_data, aes(x = time, y = surv, color = strata, fill = strata)) +
  geom_line(linewidth = 2) +
  geom_ribbon(aes(ymin = lower, ymax = upper), alpha = 0.2, color = NA) +
  scale_color_manual(values = c("#E41A1C", "#377EB8", "#9981cf", "#4DAF4A")) +
  scale_fill_manual(values = c("#E41A1C", "#377EB8", "#9981cf", "#4DAF4A")) +
  labs(
    x = "Time on platform (sec)",
    y = "Survival probability",
    color = "Platform",
    fill = "Platform"
  ) +
  theme_classic() +
  theme(
    text = element_text(size = 20),
    legend.position = "right",
    legend.justification = c(1, 1)
  )
p2 <- ggplot(km_data, aes(x = time, y = hazard, color = strata, fill = strata)) +
  geom_line(linewidth = 2) +
  geom_ribbon(aes(ymin = hazard_lower, ymax = hazard_upper), alpha = 0.2, color = NA) +
  scale_color_manual(values = c("#E41A1C", "#377EB8", "#9981cf", "#4DAF4A")) +
  scale_fill_manual(values = c("#E41A1C", "#377EB8", "#9981cf", "#4DAF4A")) +
  labs(
    x = "Time on platform (sec)",
    y = "Cumulative hazard",
    color = "Platform",
    fill = "Platform"
  ) +
  theme_classic() +
  theme(
    text = element_text(size = 20),
    legend.position = "right",
    legend.justification = c(1, 1)
  )


# Combine the two plots into one figure and save it
combined_plot <- grid.arrange(p1, p2, ncol = 2)

# Save the combined plot
ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4A.png", plot = p1, height = 120, units = "mm", dpi = 600)
ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4B.png", plot = p2, height = 120, units = "mm", dpi = 600)


## DEPRECATED: Split Dataset

the 0-75 condition has way fever number of visits than the 0-0 condition, and the 75-75 condition (this effect is already captured in transition differences). To balance the dataset, considering only the `Nth` first visits across all conditions would help interpreting the model parameters such as number of previous visits/transitions. Further, we have the assumption that the last visits are not representaive for foraging, but rather exploration or rest.

In [ ]:
ggplot(data_visits, aes(x = num_transitions, fill = condition)) +
    geom_histogram(binwidth = 1, alpha = 0.9, position = "dodge") +
    facet_wrap(~condition, scales = "free_y", ncol = 1) +
    labs(
        title = "Number of Transitions by Condition",
        x = "Number of Transitions",
        y = "Frequency",
        fill = "Condition"
    ) +
    # add quantiles for each condition
    geom_vline(data = data_visits %>% group_by(condition) %>% summarise(median = quantile(num_transitions, probs = 0.5)), 
               aes(xintercept = median), linetype = "solid", color = "black", size = 1) +
    geom_vline(data = data_visits %>% group_by(condition) %>% summarise(q75 = quantile(num_transitions, probs = 0.75)), 
               aes(xintercept = q75), linetype = "dashed", color = "black", size = 1) +
    geom_vline(data = data_visits %>% group_by(condition) %>% summarise(q25 = quantile(num_transitions, probs = 0.25)), 
               aes(xintercept = q25), linetype = "dashed", color = "black", size = 1) +
    scale_fill_brewer(palette = "Set1") +
    scale_x_continuous(breaks = seq(0, max(data_visits$num_transitions, na.rm = TRUE), by = 5)) +
    theme_classic() +
    theme(text = element_text(size = 15))

In [ ]:
# Store the result in a variable
quantiles <- data.frame(N = integer(), cond_0_0 = numeric(), cond_0_75 = numeric(), cond_75_75 = numeric())

for (N in 1:20) {
    cond_0_0 <- prop.table(table(data_visits$num_transitions < N, data_visits$condition), margin = 2)["TRUE", "0-0"]
    cond_0_75 <- prop.table(table(data_visits$num_transitions < N, data_visits$condition), margin = 2)["TRUE", "0-75"]
    cond_75_75 <- prop.table(table(data_visits$num_transitions < N, data_visits$condition), margin = 2)["TRUE", "75-75"]
    quantiles <- rbind(quantiles, data.frame(N = N, cond_0_0 = cond_0_0, cond_0_75 = cond_0_75, cond_75_75 = cond_75_75))
}

proportion <- data.frame(N = integer(), cond_0_0 = numeric(), cond_0_75 = numeric(), cond_75_75 = numeric())

for (N in 1:20) {
    cond_0_0 <- prop.table(table(data_visits$num_transitions < N, data_visits$condition), margin = 1)["TRUE", "0-0"]
    cond_0_75 <- prop.table(table(data_visits$num_transitions < N, data_visits$condition), margin = 1)["TRUE", "0-75"]
    cond_75_75 <- prop.table(table(data_visits$num_transitions < N, data_visits$condition), margin = 1)["TRUE", "75-75"]
    proportion <- rbind(proportion, data.frame(N = N, cond_0_0 = cond_0_0, cond_0_75 = cond_0_75, cond_75_75 = cond_75_75))
}

In [ ]:
k = 8
# plot quantiles
p1 <- ggplot(quantiles, aes(x = N)) +
    geom_line(aes(y = cond_0_0, color = "0-0"), size = 1) +
    geom_line(aes(y = cond_0_75, color = "0-75"), size = 1) +
    geom_line(aes(y = cond_75_75, color = "75-75"), size = 1) +
    geom_hline(yintercept = 0.5, linetype = "dashed", color = "black") +
    geom_vline(xintercept = k, linetype = "dashed", color = "black") +
    labs(
        x = "Number of Visits",
        y = "Condition Percentile",
        color = "Condition"
    ) +
    scale_color_brewer(palette = "Set1") +
    theme_classic() +
    theme(text = element_text(size = 15))

# plot proportions
p2 <- ggplot(proportion, aes(x = N)) +
    geom_line(aes(y = cond_0_0, color = "0-0"), size = 1) +
    geom_line(aes(y = cond_0_75, color = "0-75"), size = 1) +
    geom_line(aes(y = cond_75_75, color = "75-75"), size = 1) +
    geom_hline(yintercept = 0.5, linetype = "dashed", color = "black") +
    geom_hline(yintercept = 0.33, linetype = "dashed", color = "black") +
    geom_hline(yintercept = 0, linetype = "dashed", color = "black") +
    geom_vline(xintercept = k, linetype = "dashed", color = "black") +
    labs(
        x = "Number of Visits",
        y = "Sample Proportion",
        color = "Condition"
    ) +
    scale_color_brewer(palette = "Set1") +
    theme_classic() +
    theme(text = element_text(size = 15))

# combine plots side by side
grid.arrange(p1, p2, ncol = 2)

Use the first 8 visits of each condition, this would keep the proportion of visits per condition relatively stable, even if not equal. A split of `N <= 8` would represent more than half the visits in 0-75, but less than the first half visits in conditions 0-0 and 75-75. Furthermore, note that a few pigeons have less than 8 total visits, and would have no "late visits" without duplicates.

Considering only real transitions without self-transitions or censored data, pigeons perform between 2 and 80 transitions per session, and significantly less in the 0-75 condition. To split the dataset, we could either
- consider the overall median, but this would over- and under-represent the 0-75 condition in the first and last split
- consider the condition median, but this would generate an unbalanced number of visits for each condition proportional to the number of total transitions
- use the lowest median of the

## Sanity Checks

### censored data excluded in model

In [ ]:
# compare cox without consored data

### Direction of effects of single parameters

In [ ]:
# Check single effects for first visits

# travel distance has same effect as in final model
#foraging_first_cox_4.1 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + z_travel_dist + (1 | PID), data = first_visits)
# peck rate has same non-significant effect
#foraging_first_cox_4.2 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + z_peck_rate + (1 | PID), data = first_visits)
# IPI_rmssd has same strong negative effect
#foraging_first_cox_4.3 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + z_IPI_rmssd + (1 | PID), data = first_visits)
# IPD_rmssd changes direction on its own, HR = 0.81, z = -4.51, p < .001
#foraging_first_cox_4.4 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + z_IPD_rmssd + (1 | PID), data = first_visits)
# IPI_entropy has exact same effect as in final model
#foraging_first_cox_4.5 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + z_IPI_entropy + (1 | PID), data = first_visits)
# IPD_entropy has same negative effect as in final model
#foraging_first_cox_4.6 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + z_IPD_entropy + (1 | PID), data = first_visits)
# overpeck ratio has same negative effect as in final model
#foraging_first_cox_4.7 <- coxme(Surv(visit_length, status) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight + overpeck_ratio + (1 | PID), data = first_visits)

#summary(foraging_first_cox_4.7)

# 3.5 Foraging Parameters

In [ ]:
# filter real visits data without censoring
data = data_visits_finished

# Transformation of non-linear variables
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
hist(data$total_visit_length, breaks = 50)
hist(data$total_peck_count, breaks = 50)
hist(data$total_peck_rate, breaks = 50)

## Extra: Visit length

In [ ]:
# only real visits 
data = data_visits_finished
# differences in visit length by visit order and interaction with condition
foraging_model2 <- lmer(log(total_visit_length) ~ 
                cond + z_num_transitions + z_session_number_oncond + 
                z_age + z_performance + z_deprivation + z_normal_weight +
                (1 | PID) , data)

# model results
summary(foraging_model2)

# NOTE logarithmic transformation of visit length
# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(foraging_model2))
hist(residuals(foraging_model2), breaks = 50)
plot(fitted(foraging_model2), residuals(foraging_model2))

# post hoc tests for condition
emm_condition <- emmeans(foraging_model2, ~ cond)
pairwise_condition <- contrast(emm_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)

plot(emm_condition) +
    labs(
        y = "Condition",
        x = "Estimated Marginal Means (visit length)",
        color = "Comparison"
    ) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )


## Extra: Peck Count

In [ ]:
# median of overall peck count per session
abs_pecks = data$cum_peck_count + data$alt_cum_peck_count
# get the max per session * PID * condition
data$abs_pecks <- abs_pecks
data %>%
  group_by(session_number_ontask, PID, condition) %>%
  # get max peck count per session
  summarise(max_pecks = max(abs_pecks)) -> max_pecks_per_session

# get median 
max_pecks_per_session %>%
  group_by(condition) %>%
  summarise(median = quantile(max_pecks, probs = 0.5), IQR = quantile(max_pecks, probs = 0.75)-quantile(max_pecks, probs = 0.25))


In [ ]:
# correlation between peck count and visit length
cor.test(data$total_visit_length, data$total_peck_count,method = "pearson")

## Model 5: Peck Rate

In [ ]:
# why not peckrate instead of peck count?
data = data_visits_finished
foraging_model1 <- lmer(total_peck_rate ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(foraging_model1)

emms_condition <- emmeans(foraging_model1, ~ cond, type = "response")
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(foraging_model1))
hist(residuals(foraging_model1), breaks = 50)
plot(fitted(foraging_model1), residuals(foraging_model1))

plot(emms_condition) +
    labs(
        y = "Platform",
        x = "Peck-rate EMM [pecks/sec]",
    ) +
    scale_x_continuous(limits = c(0, NA), labels = scales::label_number(scale = 1)) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )
ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4D.png", height = 100, units = "mm", dpi = 600)


## Model 6: IPI RMSSD

In [ ]:
# inter peck interval rmssd with exponential distribution (exclude zeros)
data <- data_visits_finished
foraging_model2 <- lmer(total_IPI_rmssd ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)

summary(foraging_model2)

emms_condition <- emmeans(foraging_model2, ~ cond, type = "response")
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(foraging_model2))
hist(residuals(foraging_model2), breaks = 50)
plot(fitted(foraging_model2), residuals(foraging_model2))

plot(emms_condition) +
    labs(
        y = "Platform",
        x = "IPI RMSSD EMM [sec]",
    ) +
    scale_x_continuous(limits = c(0, NA), labels = scales::label_number(scale = 1)) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )

ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4E.png", height = 100, units = "mm", dpi = 600)


## Model 7: IPD RMSSD

In [ ]:
# inter peck distance RMSSD
data = data_visits_finished
foraging_model3 <- lmer(total_IPD_rmssd ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(foraging_model3)

emms_condition <- emmeans(foraging_model3, ~ cond, type = "response")
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm((residuals(foraging_model3)))
hist(scale(residuals(foraging_model3)), breaks = 50)
plot(scale(fitted(foraging_model3)), scale(residuals(foraging_model3)))

plot(emms_condition) +
    labs(
        y = "Platform",
        x = "IPD RMSSD EMM [mm]",
    ) +
    scale_x_continuous(limits = c(0, NA), labels = scales::label_number(scale = 1)) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )
ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4F.png", height = 100, units = "mm", dpi = 600)


## Model 8: Distance covered

In [ ]:
cor.test(data$total_visit_length, data$total_head_disp,method = "pearson")

In [ ]:
# head distance covered
# better residuals with log
data = data_visits_finished
foraging_model4 <- lmer(log(total_head_disp) ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(foraging_model4)

emms_condition <- emmeans(foraging_model4, ~ cond, type = "link")
#emms_condition <- emmeans(foraging_model4, ~ cond, type = "response") # for plot
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)

# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(foraging_model4))
hist(residuals(foraging_model4), breaks = 50)
plot(fitted(foraging_model4), residuals(foraging_model4))

plot(emms_condition) +
    labs(
        y = "Platform",
        x = "Area Covered EMM [m]",
    ) +
    #scale_x_continuous(limits = c(0, NA), labels = scales::label_number(scale = 0.001)) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )

ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4G.png", height = 100, units = "mm", dpi = 600)


## Model 12: Overpeck ratio

What is the difference between the overpeck ratio and the comparison of the marginal value to the habitat average?

In [ ]:
cor.test(data$total_visit_length, data$overpeck_ratio, method = "pearson")

In [ ]:
hist(data$overpeck_ratio, breaks = 50)

In [ ]:
plot(data_visits_finished$total_visit_length, data_visits_finished$overpeck_ratio, xlab = "Visit length", ylab = "Overpecking", main = "Overpecking ratio vs. visit length")

In [ ]:
data_visits_finished$visit_order_noself

In [ ]:
# for each pigeon and session, plot overpecking ratio over visits
ggplot(data_visits_finished, aes(x = visit_order_noself, y = overpeck_ratio)) +
  geom_line(aes(group = interaction(PID, session_number_ontask), color = condition), alpha = 0.5) +

  labs(
    x = "Visit number",
    y = "Overpecking ratio",
    color = "Condition"
  ) +
    theme_classic() + theme(text = element_text(size = 20)) +
    theme(aspect.ratio = 1/2)+
    scale_color_brewer(palette = "Set1")

In [ ]:
# exclude first visit with .5 overpecking
data = subset(data_visits_finished, num_transitions > 0)
foraging_model5 <- lmer(overpeck_ratio ~ cond + gmz_num_transitions + z_session_number_oncond + z_age + z_performance + z_deprivation + z_normal_weight +(1 | PID), data)
summary(foraging_model5)

emms_condition <- emmeans(foraging_model5, ~ cond, type = "response")
pairwise_condition <- contrast(emms_condition, method = 'pairwise', adjust = "holm")
summary(pairwise_condition)


# check for normality
par(mfrow=c(1,3), cex=1.5)
options(repr.plot.width=20, repr.plot.height=5)
qqnorm(residuals(foraging_model5))
hist(residuals(foraging_model5), breaks = 50)
plot(fitted(foraging_model5), residuals(foraging_model5))

plot(emms_condition) +
    labs(
        y = "Platform",
        x = "Overpecking ratio EMM "
    ) +
    theme_bw() +
    theme(
        text = element_text(size = 20),
        panel.grid.major = element_line(color = "grey", linetype = "dashed"),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        aspect.ratio = 1
    )
ggsave("C:/Users/hidalggc/sciebo/Workspace/ForagingPlatforms/figure4H.png", height = 100, units = "mm", dpi = 600)
